In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout,LSTM, Reshape
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.regularizers import l2
from keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, Dropout, Dense,LSTM

from sklearn.metrics import classification_report,confusion_matrix

import tensorflow as tf

import cv2
import os

import numpy as np






In [ ]:
base_dir = 'D:/Projects/Research/Train_test_val'

train_dir = os.path.join(base_dir, 'train')
train_dir_fire = os.path.join(train_dir, '1')
train_dir_nofire = os.path.join(train_dir, '0')

test_dir = os.path.join(base_dir, 'test')
test_dir_fire = os.path.join(test_dir, '1')
test_dir_nofire = os.path.join(test_dir, '0')

val_dir = os.path.join(base_dir, 'test')
val_dir_fire = os.path.join(val_dir, '1')
val_dir_nofire = os.path.join(val_dir, '0')

In [ ]:

train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size=(224, 224),
                                                   batch_size=32,
                                                   class_mode='binary')

test_generator = test_datagen.flow_from_directory(test_dir,
                                                   target_size=(224, 224),
                                                   batch_size=32,
                                                   class_mode='binary')

val_generator = test_datagen.flow_from_directory(val_dir,
                                                   target_size=(224, 224),
                                                   batch_size=32,
                                                   class_mode='binary')

Found 9480 images belonging to 2 classes.
Found 2033 images belonging to 2 classes.
Found 2033 images belonging to 2 classes.


In [ ]:
timesteps = 10
batch_size = 32
epochs = 20

In [ ]:
# import os
# import cv2
# import numpy as np

labels = ['Fire', 'Non-fire']
img_size = 64

def get_data(data_dir):
    data = []
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_path = os.path.join(path, img)
                img_arr = cv2.imread(img_path)[...,::-1]  # Convert BGR to RGB format
                resized_arr = cv2.resize(img_arr, (img_size, img_size))  # Reshaping images to the preferred size
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data, dtype=object)

# Example usage:
train_data = get_data(r"D:/Projects/Research/Train_test_val/train/")
val_data = get_data(r"D:/Projects/Research/Train_test_val/val/")
test_data = get_data(r"D:/Projects/Research/Train_test_val/test/")

# Now, train_data, val_data, and test_data should contain the image data and corresponding labels.


'NoneType' object is not subscriptable
'NoneType' object is not subscriptable


In [ ]:
print(train_data.shape)

(9479, 2)


In [ ]:
import os
len(os.listdir("D:/Projects/Research/Train_test_val/train/Fire"))

4667

In [ ]:
len(os.listdir("D:/Projects/Research/Train_test_val/train/Non-fire"))

4813

In [ ]:
len(os.listdir("D:/Projects/Research/Train_test_val/test/Fire"))

1001

In [ ]:
len(os.listdir("D:/Projects/Research/Train_test_val/test/Non-fire"))

1032

In [ ]:
len(os.listdir("D:/Projects/Research/Train_test_val/val/Fire"))

1000

In [ ]:
len(os.listdir("D:/Projects/Research/Train_test_val/val/Non-fire"))

1031

In [ ]:
#val
x_train = []
y_train = []
x_val = []
y_val = []
x_test = []
y_test = []

for feature, label in train_data:
  x_train.append(feature)
  y_train.append(label)

for feature, label in val_data:
  x_val.append(feature)
  y_val.append(label)

for feature, label in test_data:
  x_test.append(feature)
  y_test.append(label)

In [ ]:
# Normalize the data
x_train = np.array(x_train) / 255
x_val = np.array(x_val) / 255
x_test=np.array(x_test)/255

x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)

x_val.reshape(-1, img_size, img_size, 1)
y_val = np.array(y_val)

x_test.reshape(-1, img_size, img_size, 1)
y_test = np.array(y_test)

In [ ]:
print("x_train:",x_train.shape)
print("y_train:",y_train.shape)
print("x_test:",x_test.shape)
print("y_test:",y_test.shape)
print("x_val:",x_val.shape)
print("y_val:",y_val.shape)
print(x_train)

x_train: (9479, 64, 64, 3)
y_train: (9479,)
x_test: (2032, 64, 64, 3)
y_test: (2032,)
x_val: (2031, 64, 64, 3)
y_val: (2031,)
[[[[0.36078431 0.18039216 0.18823529]
   [0.41568627 0.18431373 0.16078431]
   [0.41568627 0.18431373 0.16078431]
   ...
   [0.4        0.17254902 0.2       ]
   [0.34509804 0.21568627 0.21176471]
   [0.27058824 0.18823529 0.22745098]]

  [[0.32156863 0.18823529 0.18823529]
   [0.35686275 0.19215686 0.16862745]
   [0.35686275 0.17647059 0.15686275]
   ...
   [0.4745098  0.19215686 0.20392157]
   [0.33333333 0.17647059 0.16862745]
   [0.30980392 0.18431373 0.2       ]]

  [[0.34901961 0.18823529 0.17254902]
   [0.36078431 0.18431373 0.15294118]
   [0.34509804 0.18431373 0.16078431]
   ...
   [0.64705882 0.20392157 0.12156863]
   [0.45882353 0.19215686 0.15686275]
   [0.4        0.2        0.20784314]]

  ...

  [[0.25882353 0.19607843 0.24705882]
   [0.26666667 0.20392157 0.25490196]
   [0.27058824 0.20784314 0.25882353]
   ...
   [0.19215686 0.16078431 0.2117647

In [ ]:
x_train = np.array(x_train).reshape(9479,1,64,64, 3)
x_test = np.array(x_test).reshape(2032, 1, 64, 64, 3)
x_val = np.array(x_val).reshape(2031, 1, 64, 64, 3)


In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(None,64,64,3))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2))
model.add(Flatten())

model.add(Reshape((1, 4096)))
model.add(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01)))


model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()


In [ ]:

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
model.layers

In [ ]:
history = model.fit(x_train,y_train,epochs = 25,validation_data = (x_val, y_val),shuffle=True,batch_size=32)

Epoch 1/25
297/297 [==============================] - 50s 150ms/step - loss: 0.3940 - accuracy: 0.8237 - val_loss: 0.3649 - val_accuracy: 0.8375
Epoch 2/25
297/297 [==============================] - 43s 144ms/step - loss: 0.3270 - accuracy: 0.8612 - val_loss: 0.3380 - val_accuracy: 0.8518
Epoch 3/25
297/297 [==============================] - 42s 143ms/step - loss: 0.2750 - accuracy: 0.8850 - val_loss: 0.3058 - val_accuracy: 0.8676
Epoch 4/25
297/297 [==============================] - 42s 143ms/step - loss: 0.2344 - accuracy: 0.9036 - val_loss: 0.3533 - val_accuracy: 0.8449
Epoch 5/25
297/297 [==============================] - 42s 143ms/step - loss: 0.1880 - accuracy: 0.9241 - val_loss: 0.3570 - val_accuracy: 0.8735
Epoch 6/25
297/297 [==============================] - 43s 143ms/step - loss: 0.1414 - accuracy: 0.9439 - val_loss: 0.3321 - val_accuracy: 0.8735
Epoch 7/25
297/297 [==============================] - 43s 143ms/step - loss: 0.0913 - accuracy: 0.9661 - val_loss: 0.3949 - val_ac

In [ ]:
predictions = model.predict(x_test)
pred=np.argmax(predictions,axis=1)
pred = pred.reshape(1,-1)[0]
print(classification_report(y_test,pred))

64/64 [==============================] - 1s 22ms/step
              precision    recall  f1-score   support

           0       0.87      0.87      0.87      1001
           1       0.87      0.87      0.87      1031

    accuracy                           0.87      2032
   macro avg       0.87      0.87      0.87      2032
weighted avg       0.87      0.87      0.87      2032



In [ ]:
model.evaluate(x_test,y_test,batch_size=32)

64/64 [==============================] - 1s 21ms/step - loss: 0.7698 - accuracy: 0.8716


[0.7697726488113403, 0.8715550899505615]

In [ ]:
#on training data
predictions = model.predict(x_train)
pred_train=np.argmax(predictions,axis=1)
pred_train = pred_train.reshape(1,-1)[0]
print(classification_report(y_train,pred_train))

297/297 [==============================] - 6s 21ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4667
           1       1.00      1.00      1.00      4812

    accuracy                           1.00      9479
   macro avg       1.00      1.00      1.00      9479
weighted avg       1.00      1.00      1.00      9479



In [ ]:
model = Sequential()
model.add(Conv2D(32,(3,3),padding="same", activation="relu", input_shape=(None,64,64,3)))

model.add(LSTM(32, return_sequences=True))
model.add(MaxPool2D((2, 2), strides=2))

model.add(Conv2D(32, (3,3), padding="same", activation="relu"))
model.add(LSTM(64, return_sequences=True))
model.add(MaxPool2D((2, 2), strides=2))

model.add(Conv2D(64, (3,3), padding="same", activation="relu"))
model.add(LSTM(128))
model.add(MaxPool2D((2, 2), strides=2))

model.add(Conv2D(64,(3,3), padding="same", activation="relu"))
model.add(MaxPool2D((2, 2), strides=2))

model.add(LSTM(32, return_sequences=True))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(128))

model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128,activation="relu"))

model.add(Dense(84,activation="relu"))
model.add(Dense(32,activation="relu"))

model.add(Dense(2, activation="softmax"))

model.summary()

ValueError: Input 0 of layer "lstm" is incompatible with the layer: expected ndim=3, found ndim=5. Full shape received: (None, None, 64, 64, 32)

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation="relu", input_shape=(None,1,64,64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation="softmax"))

ValueError: Input 0 of layer "max_pooling2d_3" is incompatible with the layer: expected ndim=4, found ndim=6. Full shape received: (None, None, 1, 62, 62, 32)

In [ ]:
# instantiate model
model = Sequential()

# add three time-distributed convolutional layers for feature extraction
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu'),input_shape=(1,150,150, 3)))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(1, 1))))

model.add(TimeDistributed(Conv2D(128, (3,3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(128, (3,3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(256, (3,3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(256, (3,3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

# extract features and dropout
model.add(TimeDistributed(Flatten()))
model.add(Dropout(0.5))

# input to LSTM
model.add(LSTM(256, return_sequences=False, dropout=0.5))

# classifier with sigmoid activation for multilabel
model.add(Dense(2, activation='sigmoid'))

# compile the model with binary_crossentropy loss for multilabel
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',metrics = ['accuracy'])

# look at the params before training
#model.summary()